In [11]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [12]:
# pip install -q -U tensorflow-addons

In [13]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [14]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [15]:
# The number of steps within one time segment
TIME_PERIODS = 20
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 20
N_FEATURES = 10

In [16]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [17]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [18]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [19]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [20]:
 pd.options.display.float_format = "{:,.5f}".format

In [21]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [34]:
verbose, epochs, batch_size = 0, 120, 1000

In [35]:
def step_decay(epoch):
   initial_lrate = 0.1
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [36]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [37]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/'

In [38]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [39]:
i=1

In [40]:
file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
print(file_path_train)
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)
file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
print(file_path_test)
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  20
n_depth=10
n_channel=10
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (9099, 20, 10)
9099 training samples
y_train shape:  (9099,)
num_time_periods 20
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (20, 10)
input_shape: (20, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:

In [46]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],20,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [47]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,10)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [48]:
if __name__ == "__main__":
    model = generate_model()

(None, 20, 10)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
ip_shape (None, 20)
ip_shape (None, 1, 20)
conv1_o (None, 1, 64)
(None, 1, 10, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 640)
(None, 52)
inputs_shape (None, 20, 10)


In [49]:
# adam=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/LSTM_nina_20X100.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 20, 10)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_40 (T [(None, 20)]         0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_42 (T [(None, 20)]         0           main[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_strided_slice_44 (T [(None, 20)]         0           main[0][0]                       
____________________________________________________________________________________

10/10 [==============================] - ETA: 0s - loss: 6.5785 - accuracy: 0.0288
Epoch 00001: val_accuracy improved from -inf to 0.04027, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 20s 2s/step - loss: 6.5785 - accuracy: 0.0288 - val_loss: 6.3850 - val_accuracy: 0.0403 - lr: 0.1000
Epoch 2/200
 9/10 [==========================>...] - ETA: 0s - loss: 6.2865 - accuracy: 0.0577
Epoch 00002: val_accuracy improved from 0.04027 to 0.13106, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 64ms/step - loss: 6.2852 - accuracy: 0.0581 - val_loss: 6.1856 - val_accuracy: 0.1311 - lr: 0.1000
Epoch 3/200
 9/10 [==========================>...] - ETA: 0s - loss: 6.0689 - accuracy: 0.1009
Epoch 00003: val_accuracy improved from 0.13106 to 0.17363, saving model to /media/naveen/nav/mat_codes/nina_

10/10 [==============================] - 1s 69ms/step - loss: 3.5611 - accuracy: 0.4821 - val_loss: 3.3567 - val_accuracy: 0.5345 - lr: 0.1000
Epoch 22/200
 9/10 [==========================>...] - ETA: 0s - loss: 3.4903 - accuracy: 0.5017
Epoch 00022: val_accuracy improved from 0.53450 to 0.53501, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 71ms/step - loss: 3.4914 - accuracy: 0.5009 - val_loss: 3.3222 - val_accuracy: 0.5350 - lr: 0.1000
Epoch 23/200
 9/10 [==========================>...] - ETA: 0s - loss: 3.4476 - accuracy: 0.4956
Epoch 00023: val_accuracy improved from 0.53501 to 0.56271, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 67ms/step - loss: 3.4496 - accuracy: 0.4953 - val_loss: 3.2488 - val_accuracy: 0.5627 - lr: 0.1000
Epoch 24/200
 9/10 [=======================

Epoch 46/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.6477 - accuracy: 0.6070
Epoch 00046: val_accuracy improved from 0.61939 to 0.62914, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 70ms/step - loss: 2.6492 - accuracy: 0.6068 - val_loss: 2.6269 - val_accuracy: 0.6291 - lr: 0.0100
Epoch 47/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.6287 - accuracy: 0.6161
Epoch 00047: val_accuracy did not improve from 0.62914
10/10 [==============================] - 0s 49ms/step - loss: 2.6309 - accuracy: 0.6158 - val_loss: 2.6229 - val_accuracy: 0.6291 - lr: 0.0100
Epoch 48/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.6016 - accuracy: 0.6216
Epoch 00048: val_accuracy did not improve from 0.62914
10/10 [==============================] - 0s 50ms/step - loss: 2.6043 - accuracy: 0.6206 - val_loss: 2.6163 - val_accuracy: 0.6250 - lr: 0.0100
Epoch

 9/10 [==========================>...] - ETA: 0s - loss: 2.4453 - accuracy: 0.6520
Epoch 00070: val_accuracy did not improve from 0.64709
10/10 [==============================] - 0s 41ms/step - loss: 2.4465 - accuracy: 0.6516 - val_loss: 2.5095 - val_accuracy: 0.6456 - lr: 0.0100
Epoch 71/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.4311 - accuracy: 0.6524
Epoch 00071: val_accuracy improved from 0.64709 to 0.65042, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 72ms/step - loss: 2.4330 - accuracy: 0.6520 - val_loss: 2.5018 - val_accuracy: 0.6504 - lr: 0.0100
Epoch 72/200
10/10 [==============================] - ETA: 0s - loss: 2.4308 - accuracy: 0.6531
Epoch 00072: val_accuracy improved from 0.65042 to 0.65401, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 70ms

Epoch 96/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3669 - accuracy: 0.6674
Epoch 00096: val_accuracy did not improve from 0.65863
10/10 [==============================] - 0s 47ms/step - loss: 2.3702 - accuracy: 0.6666 - val_loss: 2.4565 - val_accuracy: 0.6571 - lr: 0.0010
Epoch 97/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3743 - accuracy: 0.6673
Epoch 00097: val_accuracy improved from 0.65863 to 0.66171, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
10/10 [==============================] - 1s 74ms/step - loss: 2.3742 - accuracy: 0.6677 - val_loss: 2.4539 - val_accuracy: 0.6617 - lr: 0.0010
Epoch 98/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3617 - accuracy: 0.6680
Epoch 00098: val_accuracy did not improve from 0.66171
10/10 [==============================] - 0s 46ms/step - loss: 2.3648 - accuracy: 0.6671 - val_loss: 2.4537 - val_accuracy: 0.6566 - lr: 0.0010
Epoch

10/10 [==============================] - 0s 47ms/step - loss: 2.3499 - accuracy: 0.6723 - val_loss: 2.4363 - val_accuracy: 0.6589 - lr: 1.0000e-04
Epoch 124/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3509 - accuracy: 0.6699
Epoch 00124: val_accuracy did not improve from 0.66171
10/10 [==============================] - 0s 37ms/step - loss: 2.3535 - accuracy: 0.6694 - val_loss: 2.4424 - val_accuracy: 0.6607 - lr: 1.0000e-04
Epoch 125/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3662 - accuracy: 0.6633
Epoch 00125: val_accuracy did not improve from 0.66171
10/10 [==============================] - 0s 39ms/step - loss: 2.3668 - accuracy: 0.6629 - val_loss: 2.4428 - val_accuracy: 0.6568 - lr: 1.0000e-04
Epoch 126/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3529 - accuracy: 0.6708
Epoch 00126: val_accuracy did not improve from 0.66171
10/10 [==============================] - 0s 40ms/step - loss: 2.3539 - accuracy: 0.6704 - val_loss: 2.4

Epoch 150/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3441 - accuracy: 0.6751
Epoch 00150: val_accuracy did not improve from 0.66376
10/10 [==============================] - 0s 40ms/step - loss: 2.3459 - accuracy: 0.6740 - val_loss: 2.4416 - val_accuracy: 0.6586 - lr: 1.0000e-04
Epoch 151/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3486 - accuracy: 0.6720
Epoch 00151: val_accuracy did not improve from 0.66376
10/10 [==============================] - 0s 40ms/step - loss: 2.3476 - accuracy: 0.6723 - val_loss: 2.4422 - val_accuracy: 0.6617 - lr: 1.0000e-04
Epoch 152/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3376 - accuracy: 0.6733
Epoch 00152: val_accuracy did not improve from 0.66376
10/10 [==============================] - 0s 40ms/step - loss: 2.3387 - accuracy: 0.6723 - val_loss: 2.4436 - val_accuracy: 0.6607 - lr: 1.0000e-04
Epoch 153/200
10/10 [==============================] - ETA: 0s - loss: 2.3567 - accuracy: 0.6691
Epoch 

 9/10 [==========================>...] - ETA: 0s - loss: 2.3453 - accuracy: 0.6766
Epoch 00177: val_accuracy did not improve from 0.66402
10/10 [==============================] - 0s 43ms/step - loss: 2.3498 - accuracy: 0.6753 - val_loss: 2.4436 - val_accuracy: 0.6579 - lr: 1.0000e-05
Epoch 178/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3598 - accuracy: 0.6648
Epoch 00178: val_accuracy did not improve from 0.66402
10/10 [==============================] - 0s 43ms/step - loss: 2.3586 - accuracy: 0.6652 - val_loss: 2.4461 - val_accuracy: 0.6597 - lr: 1.0000e-05
Epoch 179/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3468 - accuracy: 0.6680
Epoch 00179: val_accuracy did not improve from 0.66402
10/10 [==============================] - 0s 37ms/step - loss: 2.3485 - accuracy: 0.6673 - val_loss: 2.4482 - val_accuracy: 0.6576 - lr: 1.0000e-05
Epoch 180/200
 9/10 [==========================>...] - ETA: 0s - loss: 2.3477 - accuracy: 0.6704
Epoch 00180: val_acc

In [45]:
statistics.mean(test_acc)

0.7612208127975464